In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical
from keras.models import Model, Sequential, load_model
from keras.layers import CuDNNLSTM, Dense, Input, Dropout, Activation, Concatenate
from keras.optimizers import Adam

from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.utils import multi_gpu_model
from tqdm import tqdm
from scipy.special import softmax
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
class_weights = [1.09518711e+01, 3.07683547e+00, 3.58078232e-01, 2.61727260e-01,
       4.38833908e-01, 1.35073077e+03, 6.72563039e+00, 9.89546351e+00,
       3.99230769e+00, 1.03902367e+01, 4.49295078e-01, 4.26546559e+01,
       1.18329459e+00]

In [3]:
demo = pd.read_csv('dataset/demo.csv')
demo = demo.fillna(0)
demo['c1'] = demo['c1'].astype('int')

txn = pd.read_csv('dataset/txn.csv')
_txn = txn.copy()
txn.rename({
    'n3':'date_idx'
}, axis=1, inplace=True)
txn.drop('t0', axis=1, inplace=True)

train_set = pd.read_csv('dataset/train.csv')
test_set = pd.read_csv('dataset/test.csv')

scaler = StandardScaler()

In [4]:
id_ccno = txn[['id', 'old_cc_no']].drop_duplicates()
train_id_ccno = pd.merge(id_ccno, train_set, on='id')

In [5]:
txn_n = txn[['id', 'old_cc_no', 'date_idx', 'n4', 'n5', 'n6', 'n7']].copy()
txn_n['count'] = 1

In [6]:
LENGTH_SEQ = 365
LENGTH_SEQ = 53

In [7]:
txn_n['date_idx'] = txn_n['date_idx'] // 7 + 1 # WEEKLY
txn_n = txn_n.groupby(['id', 'old_cc_no', 'date_idx']).sum()

In [25]:
new_txn_n = txn_n.reset_index().groupby(['id', 'old_cc_no']).count()

In [26]:
new_txn_n.head(20)

date_idx  n4  n5  n6  n7  count
id old_cc_no                                 
1  7051             26  26  26  26  26     26
   7052             33  33  33  33  33     33
2  126612           53  53  53  53  53     53
   126613           16  16  16  16  16     16
3  6404             31  31  31  31  31     31
   6405             48  48  48  48  48     48
   6406             40  40  40  40  40     40
4  266126           38  38  38  38  38     38
5  72072            49  49  49  49  49     49
6  122485            1   1   1   1   1      1
   122486           52  52  52  52  52     52
7  292175           42  42  42  42  42     42
8  289017           50  50  50  50  50     50
9  205237           53  53  53  53  53     53
10 2522             45  45  45  45  45     45
   2524             53  53  53  53  53     53
11 26825            24  24  24  24  24     24
   26826            48  48  48  48  48     48
   26827            21  21  21  21  21     21
12 12382            49  49  49  49  49     49

In [30]:
new_txn_n.sort_values('count', ascending=False)

,,date_idx,n4,n5,n6,n7,count
id,old_cc_no,,,,,,
4710,275052,53,53,53,53,53,53
1074,29688,53,53,53,53,53,53
1076,184355,53,53,53,53,53,53
3124,126043,53,53,53,53,53,53
1078,206567,53,53,53,53,53,53
24237,60149,53,53,53,53,53,53
1081,225944,53,53,53,53,53,53
6242,106893,53,53,53,53,53,53
1082,256978,53,53,53,53,53,53


In [28]:
max_txn_n

,,date_idx,n4,n5,n6,n7,count
id,old_cc_no,,,,,,
4710,275052,53,53,53,53,53,53
6940,37674,52,52,52,52,52,52
15409,5127,51,51,51,51,51,51
11599,168715,50,50,50,50,50,50
7287,163735,49,49,49,49,49,49
10714,119919,48,48,48,48,48,48
19047,251778,47,47,47,47,47,47
17909,276047,46,46,46,46,46,46
16773,31828,45,45,45,45,45,45


In [8]:
scaled_txn_n = txn_n

In [9]:
scaled_txn_n.head()

n4    n5     n6     n7  count
id old_cc_no date_idx                                  
1  7051      1         89000  5181  25825  11475     51
             2         25000  4286  20996   9450     42
             3         53500  4155  20589   9225     41
             4         28300  2562  12798   5625     25
             5         18000  3284  16822   7200     32

In [10]:
txn_scaler = StandardScaler()
scaled_txn_n[['n4', 'n5', 'n6', 'n7', 'count']] = txn_scaler.fit_transform(txn_n)

In [11]:
scaled_txn_n = scaled_txn_n.reset_index().set_index('old_cc_no')

In [12]:
txn[['old_cc_label', 'c5', 'c6', 'c7']].head()

,old_cc_label,c5,c6,c7
0,10,14,10,58
1,10,11,-10,53
2,10,12,-10,58
3,10,11,-10,80
4,10,12,11,58


In [13]:
txn.head()

,id,old_cc_no,old_cc_label,c5,c6,c7,date_idx,n4,n5,n6,n7
0,11592,256451,10,14,10,58,296,500,159,600,3801
1,11592,256451,10,11,-10,53,218,500,97,800,3801
2,11592,256451,10,12,-10,58,133,700,101,300,3801
3,11592,256451,10,11,-10,80,340,4000,95,0,3801
4,11592,256451,10,12,11,58,306,400,124,300,3801


In [242]:
def prepare_demo_data(ids, is_test=False):
    filtered_demo = demo.set_index('id').loc[ids]
    filtered_txn = txn.set_index('id').loc[ids]
    sum_amount = filtered_txn.reset_index()[['id', 'n6']].groupby('id').sum().to_numpy()
    old_cc_label_onehot = to_categorical(filtered_txn.groupby('id')['old_cc_label'].first(), 14)
    demo_data = filtered_demo[['n0', 'n1', 'n2']].to_numpy()
    job_onehot = to_categorical(filtered_demo['c1'].to_numpy(), 14)
    x = np.concatenate((job_onehot, old_cc_label_onehot, demo_data, sum_amount), axis=1)
    
    if is_test:
        return x
    
    x[:, 28:] = scaler.fit_transform(x[:, 28:])
    
    return x, to_categorical(train_set.set_index('id').loc[ids]['label'].to_numpy(), 13)

In [257]:
def preprocess_credit_class(old_cc_no_ids):

    txn_joined = _txn.set_index('old_cc_no')
    txn_joined = txn_joined.loc[old_cc_no_ids]

    txn_joined = txn_joined.drop(['n3', 'n4', 'n5', 'n6', 'n7', 't0'], axis=1)
    txn_joined = txn_joined.reset_index()

    _x = txn_joined[['old_cc_no','old_cc_label', 'c5', 'c6', 'c7']]


    dfs = []
    for (_, old_cc_no, old_cc_label, c5, c6, c7) in tqdm(_x.itertuples()):
        old_cc_label = to_categorical(old_cc_label, 13)
        c5 = to_categorical(c5, 100)
        c6 = to_categorical(c6, 79)
        c7 = to_categorical(c7, 95)
        onehot = np.concatenate((old_cc_label, c5, c6, c7))

        df = pd.DataFrame({
            'old_cc_no': old_cc_no,
        }, index=[old_cc_no])
        df['onehot'] = [onehot]
        dfs.append(df)
    return pd.concat(dfs,ignore_index=False)

In [258]:
padding_value = txn_scaler.transform([[0, 0, 0, 0, 0]])
def create_seq_from_group(group):
    seq = np.ones((LENGTH_SEQ, 5)) * padding_value
    group = group.sort_values('date_idx')
    for (_, _, date_idx, n4, n5, n6, n7, count) in group.itertuples():
        seq[int(date_idx) - 1] = [n4, n5, n6, n7, count]
    return seq

def create_seq(txn):
    grouped = txn.groupby(['old_cc_no'])
    seqs = []
    for name, group in tqdm(grouped):
        seq = create_seq_from_group(group)
        seqs.append(seq)
    seqs = np.array(seqs)
    return seqs

def create_seq_dataframe(old_cc_nos):

    txn = scaled_txn_n.loc[old_cc_nos]
    txn = txn.sort_values(['old_cc_no', 'date_idx'])
    grouped = txn.groupby(['old_cc_no'])
    output_df = pd.DataFrame(columns=['old_cc_no', 'seq'])
    dfs = []
    for old_cc_no, group in tqdm(grouped):
        seq = create_seq_from_group(group)
#         output_df = output_df.append({
#             'old_cc_no': old_cc_no,
#             'seq': seq
#         }, ignore_index=True, )
        df = pd.DataFrame({
            'old_cc_no': old_cc_no,
        }, index=[old_cc_no])
        df['seq'] = [seq]
        dfs.append(df)
    
#     return output_df
    return pd.concat(dfs, ignore_index=False)

In [259]:
def create_test_dataframe(ids):
#     ids = test_set.iloc[:500]['id']
    test_df = pd.merge(id_ccno, ids, on='id')

    cc_seq = create_seq_dataframe(test_df['old_cc_no'])
    cc_class = preprocess_credit_class(test_df['old_cc_no'])

    test_df = pd.merge(test_df, cc_seq, on='old_cc_no')
    test_df = pd.merge(test_df, cc_class, on='old_cc_no')
    return test_df

In [260]:
batch_size = 32
def create_data(ids, is_test=False):
    
    train_old_cc_nos = pd.merge(train_id_ccno, ids, on='id')['old_cc_no']

    train_txn = scaled_txn_n.loc[train_old_cc_nos]
    train_txn = train_txn.sort_values(['old_cc_no', 'date_idx'])

    seqs = create_seq(train_txn)
#     return seqs
    onehot = preprocess_credit_class(train_old_cc_nos.to_list())
    onehot = np.array([v for v in onehot['onehot'].to_numpy()])
    

    demo_x, _ = prepare_demo_data(ids, False)
    
    if is_test:
        return onehot, demo_x, seqs
    else:
        labels = train_id_ccno.set_index('old_cc_no').loc[train_old_cc_nos].sort_values('old_cc_no')['label']
        labels = np.array([to_categorical(label, 13) if not np.isnan(label) else to_categorical(0, 13) for label in labels])
        return onehot, demo_x, seqs, labels

In [3]:
seqs = np.load('data_saved/seq-weekly.npy')
labels = np.load('data_saved/label-weekly.npy')

In [ ]:
ids = train_set['id']
onehot, demo_x, seqs, labels = create_data(ids)
# train_old_cc_nos = create_data(ids)



  0%|          | 0/150511 [00:00<?, ?it/s]

  0%|          | 1/150511 [00:00<4:30:58,  9.26it/s]

  0%|          | 76/150511 [00:00<3:10:35, 13.15it/s]

  0%|          | 151/150511 [00:00<2:14:21, 18.65it/s]

  0%|          | 227/150511 [00:00<1:34:59, 26.37it/s]

  0%|          | 303/150511 [00:00<1:07:27, 37.11it/s]

  0%|          | 379/150511 [00:00<48:11, 51.92it/s]  

  0%|          | 454/150511 [00:00<34:43, 72.01it/s]

  0%|          | 530/150511 [00:00<25:17, 98.82it/s]

  0%|          | 606/150511 [00:00<18:41, 133.68it/s]

  0%|          | 682/150511 [00:01<14:04, 177.43it/s]

  1%|          | 758/150511 [00:01<10:50, 230.25it/s]

  1%|          | 834/150511 [00:01<08:34, 290.88it/s]

  1%|          | 910/150511 [00:01<06:59, 356.41it/s]

  1%|          | 986/150511 [00:01<05:53, 423.14it/s]

  1%|          | 1062/150511 [00:01<05:06, 487.52it/s]

  1%|          | 1138/150511 [00:01<04:33, 545.39it/s]

  1%|          | 1214/150511 [00:01<04:11, 594.60it/s]

  1%|          

 13%|█▎        | 19656/150511 [00:26<02:57, 738.09it/s]

 13%|█▎        | 19731/150511 [00:26<02:56, 739.67it/s]

 13%|█▎        | 19806/150511 [00:26<02:56, 741.83it/s]

 13%|█▎        | 19881/150511 [00:26<02:55, 743.00it/s]

 13%|█▎        | 19956/150511 [00:26<02:55, 743.12it/s]

 13%|█▎        | 20031/150511 [00:26<02:55, 742.81it/s]

 13%|█▎        | 20106/150511 [00:27<02:55, 744.14it/s]

 13%|█▎        | 20181/150511 [00:27<02:54, 744.84it/s]

 13%|█▎        | 20256/150511 [00:27<02:54, 744.94it/s]

 14%|█▎        | 20331/150511 [00:27<02:54, 745.09it/s]

 14%|█▎        | 20406/150511 [00:27<02:54, 744.85it/s]

 14%|█▎        | 20481/150511 [00:27<02:54, 744.32it/s]

 14%|█▎        | 20556/150511 [00:27<02:54, 745.58it/s]

 14%|█▎        | 20631/150511 [00:27<02:54, 745.91it/s]

 14%|█▍        | 20706/150511 [00:27<02:53, 746.04it/s]

 14%|█▍        | 20781/150511 [00:27<02:54, 745.12it/s]

 14%|█▍        | 20856/150511 [00:28<02:53, 745.39it/s]

 14%|█▍        | 20931/150511 [

 26%|██▌       | 39175/150511 [00:52<02:29, 744.77it/s]

 26%|██▌       | 39250/150511 [00:52<02:29, 745.09it/s]

 26%|██▌       | 39325/150511 [00:52<02:28, 746.35it/s]

 26%|██▌       | 39400/150511 [00:52<02:28, 745.88it/s]

 26%|██▌       | 39475/150511 [00:53<02:29, 745.10it/s]

 26%|██▋       | 39550/150511 [00:53<02:29, 744.06it/s]

 26%|██▋       | 39625/150511 [00:53<02:28, 744.73it/s]

 26%|██▋       | 39700/150511 [00:53<02:28, 745.87it/s]

 26%|██▋       | 39775/150511 [00:53<02:28, 746.43it/s]

 26%|██▋       | 39850/150511 [00:53<02:28, 745.69it/s]

 27%|██▋       | 39925/150511 [00:53<02:28, 745.71it/s]

 27%|██▋       | 40000/150511 [00:53<02:28, 745.08it/s]

 27%|██▋       | 40075/150511 [00:53<02:27, 746.39it/s]

 27%|██▋       | 40150/150511 [00:53<02:27, 746.24it/s]

 27%|██▋       | 40225/150511 [00:54<02:27, 745.24it/s]

 27%|██▋       | 40300/150511 [00:54<02:27, 745.80it/s]

 27%|██▋       | 40375/150511 [00:54<02:27, 746.32it/s]

 27%|██▋       | 40450/150511 [

 39%|███▉      | 58688/150511 [01:18<02:03, 746.45it/s]

 39%|███▉      | 58763/150511 [01:18<02:02, 746.30it/s]

 39%|███▉      | 58838/150511 [01:19<02:02, 746.96it/s]

 39%|███▉      | 58913/150511 [01:19<02:02, 746.58it/s]

 39%|███▉      | 58988/150511 [01:19<02:02, 745.94it/s]

 39%|███▉      | 59063/150511 [01:19<02:02, 745.10it/s]

 39%|███▉      | 59138/150511 [01:19<02:02, 744.07it/s]

 39%|███▉      | 59213/150511 [01:19<02:02, 745.48it/s]

 39%|███▉      | 59288/150511 [01:19<02:02, 745.05it/s]

 39%|███▉      | 59363/150511 [01:19<02:02, 744.38it/s]

 39%|███▉      | 59438/150511 [01:19<02:02, 744.01it/s]

 40%|███▉      | 59513/150511 [01:19<02:02, 744.18it/s]

 40%|███▉      | 59588/150511 [01:20<02:03, 736.83it/s]

 40%|███▉      | 59663/150511 [01:20<02:02, 740.07it/s]

 40%|███▉      | 59738/150511 [01:20<02:02, 742.52it/s]

 40%|███▉      | 59813/150511 [01:20<02:02, 741.23it/s]

 40%|███▉      | 59888/150511 [01:20<02:01, 743.39it/s]

 40%|███▉      | 59963/150511 [

 52%|█████▏    | 78190/150511 [01:45<01:37, 739.51it/s]

 52%|█████▏    | 78265/150511 [01:45<01:37, 740.19it/s]

 52%|█████▏    | 78340/150511 [01:45<01:37, 741.70it/s]

 52%|█████▏    | 78415/150511 [01:45<01:37, 742.16it/s]

 52%|█████▏    | 78490/150511 [01:45<01:36, 743.81it/s]

 52%|█████▏    | 78565/150511 [01:45<01:36, 743.53it/s]

 52%|█████▏    | 78640/150511 [01:45<01:36, 743.13it/s]

 52%|█████▏    | 78715/150511 [01:45<01:36, 742.17it/s]

 52%|█████▏    | 78790/150511 [01:45<01:36, 741.76it/s]

 52%|█████▏    | 78865/150511 [01:46<01:36, 741.24it/s]

 52%|█████▏    | 78940/150511 [01:46<01:36, 742.03it/s]

 52%|█████▏    | 79015/150511 [01:46<01:36, 742.40it/s]

 53%|█████▎    | 79090/150511 [01:46<01:36, 742.36it/s]

 53%|█████▎    | 79165/150511 [01:46<01:36, 742.16it/s]

 53%|█████▎    | 79240/150511 [01:46<01:36, 741.87it/s]

 53%|█████▎    | 79315/150511 [01:46<01:35, 742.53it/s]

 53%|█████▎    | 79390/150511 [01:46<01:35, 742.89it/s]

 53%|█████▎    | 79465/150511 [

...

11101912it [2:25:07, 1304.94it/s]

11102043it [2:25:07, 1306.18it/s]

11102174it [2:25:07, 1305.12it/s]

11102305it [2:25:07, 1302.60it/s]

11102436it [2:25:07, 1302.71it/s]

11102567it [2:25:08, 1302.36it/s]

11102698it [2:25:08, 1302.29it/s]

11102829it [2:25:08, 1302.96it/s]

11102960it [2:25:08, 1302.52it/s]

11103091it [2:25:08, 1304.17it/s]

11103222it [2:25:08, 1298.43it/s]

11103353it [2:25:08, 1300.30it/s]

11103485it [2:25:08, 1303.42it/s]

11103616it [2:25:08, 1302.76it/s]

11103747it [2:25:08, 1304.32it/s]

11103878it [2:25:09, 1305.13it/s]

11104009it [2:25:09, 1302.28it/s]

11104140it [2:25:09, 1290.46it/s]

11104271it [2:25:09, 1295.19it/s]

11104402it [2:25:09, 1297.39it/s]

11104533it [2:25:09, 1299.92it/s]

11104665it [2:25:09, 1303.23it/s]

11104796it [2:25:09, 1301.90it/s]

11104927it [2:25:09, 1304.02it/s]

11105058it [2:25:09, 1305.11it/s]

11105189it [2:25:10, 1303.75it/s]

11105320it [2:25:10, 1303.72it/s]

11105451it [2:25:10, 1302.47it/s]

11105582it [2:25:10,

11153803it [2:25:47, 1301.63it/s]

11153934it [2:25:47, 1302.66it/s]

11154065it [2:25:47, 1303.23it/s]

11154196it [2:25:48, 1303.16it/s]

11154327it [2:25:48, 1302.01it/s]

11154458it [2:25:48, 1302.86it/s]

11154589it [2:25:48, 1296.41it/s]

11154720it [2:25:48, 1297.58it/s]

11154852it [2:25:48, 1301.13it/s]

11154983it [2:25:48, 1301.68it/s]

11155114it [2:25:48, 1303.28it/s]

11155245it [2:25:48, 1304.22it/s]

11155376it [2:25:48, 1304.35it/s]

11155507it [2:25:49, 1301.84it/s]

11155638it [2:25:49, 1302.07it/s]

11155769it [2:25:49, 1300.65it/s]

11155900it [2:25:49, 1302.84it/s]

11156031it [2:25:49, 1304.40it/s]

11156162it [2:25:49, 1303.46it/s]

11156293it [2:25:49, 1304.52it/s]

11156424it [2:25:49, 1297.39it/s]

11156555it [2:25:49, 1298.93it/s]

11156686it [2:25:49, 1301.48it/s]

11156817it [2:25:50, 1303.24it/s]

11156948it [2:25:50, 1303.72it/s]

11157079it [2:25:50, 1304.47it/s]

11157210it [2:25:50, 1305.96it/s]

11157341it [2:25:50, 1284.34it/s]

11157472it [2:25:50,

11205764it [2:26:27, 1292.59it/s]

11205895it [2:26:27, 1295.24it/s]

11206025it [2:26:28, 1293.89it/s]

11206156it [2:26:28, 1295.72it/s]

11206286it [2:26:28, 1295.92it/s]

11206416it [2:26:28, 1288.08it/s]

11206546it [2:26:28, 1291.51it/s]

11206676it [2:26:28, 1293.41it/s]

11206806it [2:26:28, 1294.15it/s]

11206936it [2:26:28, 1295.56it/s]

11207067it [2:26:28, 1297.56it/s]

11207197it [2:26:28, 1295.95it/s]

11207327it [2:26:29, 1273.26it/s]

11207458it [2:26:29, 1281.60it/s]

11207588it [2:26:29, 1284.11it/s]

11207718it [2:26:29, 1286.65it/s]

11207849it [2:26:29, 1290.72it/s]

11207979it [2:26:29, 1292.23it/s]

11208110it [2:26:29, 1295.75it/s]

11208240it [2:26:29, 1295.58it/s]

11208370it [2:26:29, 1295.57it/s]

11208500it [2:26:29, 1274.90it/s]

11208630it [2:26:30, 1281.09it/s]

11208761it [2:26:30, 1286.73it/s]

11208891it [2:26:30, 1289.61it/s]

11209021it [2:26:30, 1292.46it/s]

11209152it [2:26:30, 1294.80it/s]

11209283it [2:26:30, 1297.10it/s]

11209413it [2:26:30,

11731884it [2:33:14, 1304.07it/s]

11732015it [2:33:14, 1298.80it/s]

11732146it [2:33:14, 1300.09it/s]

11732278it [2:33:14, 1303.51it/s]

11732409it [2:33:14, 1303.43it/s]

11732540it [2:33:14, 1304.22it/s]

11732671it [2:33:14, 1305.44it/s]

11732802it [2:33:14, 1304.29it/s]

11732933it [2:33:15, 1293.33it/s]

11733063it [2:33:15, 1295.22it/s]

11733195it [2:33:15, 1299.72it/s]

11733326it [2:33:15, 1302.32it/s]

11733457it [2:33:15, 1302.71it/s]

11733588it [2:33:15, 1304.67it/s]

11733719it [2:33:15, 1306.19it/s]

11733850it [2:33:15, 1300.95it/s]

11733981it [2:33:15, 1300.09it/s]

11734112it [2:33:15, 1302.00it/s]

11734243it [2:33:16, 1300.68it/s]

11734374it [2:33:16, 1302.18it/s]

11734505it [2:33:16, 1303.71it/s]

11734636it [2:33:16, 1303.19it/s]

11734767it [2:33:16, 1272.97it/s]

11734898it [2:33:16, 1282.27it/s]

11735029it [2:33:16, 1288.36it/s]

11735160it [2:33:16, 1293.05it/s]

11735291it [2:33:16, 1295.28it/s]

11735422it [2:33:16, 1298.49it/s]

11735553it [2:33:17,

11783781it [2:33:54, 1299.82it/s]

11783912it [2:33:54, 1302.57it/s]

11784043it [2:33:54, 1303.71it/s]

11784174it [2:33:54, 1303.00it/s]

11784305it [2:33:54, 1304.00it/s]

11784436it [2:33:54, 1304.92it/s]

11784567it [2:33:55, 1297.95it/s]

11784698it [2:33:55, 1299.05it/s]

11784829it [2:33:55, 1299.72it/s]

11784961it [2:33:55, 1302.85it/s]

11785092it [2:33:55, 1304.43it/s]

11785223it [2:33:55, 1303.59it/s]

11785354it [2:33:55, 1304.66it/s]

11785485it [2:33:55, 1305.95it/s]

11785616it [2:33:55, 1295.38it/s]

11785747it [2:33:55, 1298.31it/s]

11785878it [2:33:56, 1300.98it/s]

11786009it [2:33:56, 1299.61it/s]

11786141it [2:33:56, 1302.52it/s]

11786272it [2:33:56, 1304.23it/s]

11786403it [2:33:56, 1304.28it/s]

11786534it [2:33:56, 1296.92it/s]

11786665it [2:33:56, 1298.77it/s]

11786795it [2:33:56, 1298.61it/s]

11786927it [2:33:56, 1301.90it/s]

11787058it [2:33:56, 1301.68it/s]

11787189it [2:33:57, 1301.35it/s]

11787320it [2:33:57, 1301.68it/s]

11787451it [2:33:57,

In [195]:
onehot = np.load('data_saved/class.npy')
demo_x = np.load('data_saved/demo_x.npy')
seqs = np.load('data_saved/seq-weekly.npy')
labels = np.load('data_saved/label-weekly.npy')

In [ ]:
onehot.shape, demo_x.shape, seqs.shape, labels.shape

In [252]:
np.save('data_saved/class.npy', onehot)
np.save('data_saved/demo_x.npy', demo_x)
np.save('data_saved/seq-weekly.npy', seqs)
np.save('data_saved/label-weekly.npy', labels)


In [254]:
split_rate = 0.2
demo_train, demo_test, onehot_train, onehot_test,seq_train, seq_test, label_train, label_test = train_test_split(demo_x, onehot, seqs, labels, test_size=split_rate, random_state=42)



ValueError: Found input variables with inconsistent numbers of samples: [100000, 150511, 150511, 150511]

In [256]:
seqs.shape

(150511, 53, 5)

In [212]:
def build_class_model(n_input):
    model = Sequential()
    
    model.add(Dense(128, activation='relu', input_shape=(n_input, )))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(13, activation='softmax'))
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model


In [210]:
def build_txn_model():
    model = Sequential()
    model.add(CuDNNLSTM(32, return_sequences=True, input_shape=(LENGTH_SEQ, 5)))
#     model.add(CuDNNLSTM(32, return_sequences=True))
#     model.add(CuDNNLSTM(32, return_sequences=True))
    model.add(CuDNNLSTM(32))
    model.add(Dense(13, activation='softmax'))
    model.summary()
    # model = multi_gpu_model(model, gpus=2)
    model.compile(optimizer=Adam(),
                  loss='categorical_crossentropy',
                 metrics=['acc'])
    return model



In [ ]:
def build_demo_cc_model():
    model = Sequential()

    model.add(Dense(64, activation='relu', input_shape=(34, )))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(13, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [206]:
class_model = build_class_model(287)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 128)               36864     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_9 (Dense)              (None, 16)                1040      
_________________________________________________________________
dense_10 (Dense)             (None, 13)               

In [207]:
class_model.fit(onehot_train, label_train,
                validation_data=(onehot_test, label_test),
                class_weight=class_weights,
                epochs=100
               )

Train on 120408 samples, validate on 30103 samples
Epoch 1/100
120408/120408 [==============================] - 10s 80us/step - loss: 1.8325 - accuracy: 0.2916 - val_loss: 1.8124 - val_accuracy: 0.2948
Epoch 2/100
120408/120408 [==============================] - 9s 78us/step - loss: 1.8196 - accuracy: 0.2938 - val_loss: 1.8137 - val_accuracy: 0.2948
Epoch 3/100
120408/120408 [==============================] - 9s 77us/step - loss: 1.8182 - accuracy: 0.2938 - val_loss: 1.8142 - val_accuracy: 0.2948
Epoch 4/100
120408/120408 [==============================] - 9s 77us/step - loss: 1.8177 - accuracy: 0.2938 - val_loss: 1.8129 - val_accuracy: 0.2948
Epoch 5/100
120408/120408 [==============================] - 9s 78us/step - loss: 1.8172 - accuracy: 0.2939 - val_loss: 1.8125 - val_accuracy: 0.2948
Epoch 6/100
120408/120408 [==============================] - 9s 77us/step - loss: 1.8168 - accuracy: 0.2938 - val_loss: 1.8131 - val_accuracy: 0.2948
Epoch 7/100
120408/120408 [=====================

KeyboardInterrupt: 

In [213]:
txn_model = build_txn_model()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_3 (CuDNNLSTM)     (None, 53, 32)            4992      
_________________________________________________________________
cu_dnnlstm_4 (CuDNNLSTM)     (None, 32)                8448      
_________________________________________________________________
dense_12 (Dense)             (None, 13)                429       
Total params: 13,869
Trainable params: 13,869
Non-trainable params: 0
_________________________________________________________________


In [214]:
txn_model.fit(seq_train, label_train,
             validation_data=(seq_test, label_test),
             class_weight=class_weights,
             epochs=100
             )

Train on 120408 samples, validate on 30103 samples
Epoch 1/100
120408/120408 [==============================] - 24s 196us/step - loss: 1.7493 - acc: 0.3382 - val_loss: 1.7061 - val_acc: 0.3539
Epoch 2/100
120408/120408 [==============================] - 23s 194us/step - loss: 1.6999 - acc: 0.3621 - val_loss: 1.6915 - val_acc: 0.3669
Epoch 3/100
120408/120408 [==============================] - 23s 191us/step - loss: 1.6910 - acc: 0.3650 - val_loss: 1.6854 - val_acc: 0.3695
Epoch 4/100
120408/120408 [==============================] - 23s 191us/step - loss: 1.6828 - acc: 0.3691 - val_loss: 1.6801 - val_acc: 0.3649
Epoch 5/100
120408/120408 [==============================] - 23s 191us/step - loss: 1.6786 - acc: 0.3702 - val_loss: 1.6783 - val_acc: 0.3694
Epoch 6/100
120408/120408 [==============================] - 23s 191us/step - loss: 1.6752 - acc: 0.3727 - val_loss: 1.6719 - val_acc: 0.3738
Epoch 7/100
120408/120408 [==============================] - 23s 192us/step - loss: 1.6709 - acc:

KeyboardInterrupt: 

In [216]:
x = Concatenate()([class_model.layers[-2].output, txn_model.layers[-2].output])
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)
x = Dense(13, activation='softmax')(x)
model = Model(inputs=[class_model.input, txn_model.input], output = x)
# model = multi_gpu_model(model, gpus=3)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
dense_6_input (InputLayer)      (None, 287)          0                                            
__________________________________________________________________________________________________
dense_6 (Dense)                 (None, 128)          36864       dense_6_input[0][0]              
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 128)          0           dense_6[0][0]                    
__________________________________________________________________________________________________
dense_7 (Dense)                 (None, 64)           8256        dropout_3[0][0]                  
____________________________________________________________________________________________

/home/porlolicon/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`
  """


In [218]:
for layer in class_model.layers:
    layer.trainable = False
for layer in txn_model.layers:
    layer.trainable = False

In [219]:
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
             metrics=['acc'])

In [1]:
model.summary()

NameError: name 'model' is not defined

In [95]:
model = load_model('weights/ensemble/ep023-val_loss:1.6206224270198957-val_acc:0.38749611377716064.hdf5')

W1201 01:53:24.090511 140155359631168 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
W1201 01:53:26.442753 140155359631168 module_wrapper.py:137] From /home/porlolicon/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



In [222]:


log_dir = 'weights/ensemble2'
model.fit([onehot, seqs],
          labels,
          batch_size=128,
          shuffle=True,
          class_weight=class_weights,
          validation_split=0.3,
          callbacks=[
              TensorBoard(log_dir),
              ModelCheckpoint(log_dir+"/ep{epoch:03d}-val_loss:{val_loss}-val_acc:{val_acc}.hdf5", 
                             monitor='val_loss',
                              mode='min',
                              save_best_only=True,
                             )
          ],
          initial_epoch=0,
          epochs=100)

W1201 03:32:19.369080 140155359631168 module_wrapper.py:137] From /home/porlolicon/.local/lib/python3.6/site-packages/keras/callbacks/tensorboard_v1.py:200: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.



Train on 105357 samples, validate on 45154 samples
Epoch 1/100
105357/105357 [==============================] - 6s 55us/step - loss: 1.7104 - acc: 0.3719 - val_loss: 1.5842 - val_acc: 0.4050
Epoch 2/100
105357/105357 [==============================] - 6s 53us/step - loss: 1.6579 - acc: 0.3844 - val_loss: 1.5784 - val_acc: 0.4071
Epoch 3/100
105357/105357 [==============================] - 6s 53us/step - loss: 1.6551 - acc: 0.3851 - val_loss: 1.5771 - val_acc: 0.4066
Epoch 4/100
105357/105357 [==============================] - 6s 53us/step - loss: 1.6541 - acc: 0.3852 - val_loss: 1.5763 - val_acc: 0.4061
Epoch 5/100
105357/105357 [==============================] - 6s 53us/step - loss: 1.6522 - acc: 0.3858 - val_loss: 1.5780 - val_acc: 0.4050
Epoch 6/100
105357/105357 [==============================] - 6s 52us/step - loss: 1.6521 - acc: 0.3862 - val_loss: 1.5736 - val_acc: 0.4078
Epoch 7/100
105357/105357 [==============================] - 6s 53us/step - loss: 1.6511 - acc: 0.3861 - val_

Epoch 59/100
105357/105357 [==============================] - 4s 39us/step - loss: 1.6386 - acc: 0.3896 - val_loss: 1.5822 - val_acc: 0.4036
Epoch 60/100
105357/105357 [==============================] - 4s 39us/step - loss: 1.6383 - acc: 0.3890 - val_loss: 1.5771 - val_acc: 0.4063
Epoch 61/100
105357/105357 [==============================] - 4s 38us/step - loss: 1.6380 - acc: 0.3887 - val_loss: 1.5815 - val_acc: 0.4051
Epoch 62/100
105357/105357 [==============================] - 4s 38us/step - loss: 1.6381 - acc: 0.3884 - val_loss: 1.5805 - val_acc: 0.4048
Epoch 63/100
105357/105357 [==============================] - 4s 38us/step - loss: 1.6376 - acc: 0.3891 - val_loss: 1.5783 - val_acc: 0.4047
Epoch 64/100
105357/105357 [==============================] - 4s 38us/step - loss: 1.6372 - acc: 0.3894 - val_loss: 1.5795 - val_acc: 0.4067
Epoch 65/100
105357/105357 [==============================] - 4s 39us/step - loss: 1.6373 - acc: 0.3892 - val_loss: 1.5800 - val_acc: 0.4045
Epoch 66/100


KeyboardInterrupt: 

In [223]:
ids = test_set['id']
test_df = create_test_dataframe(ids)

 10%|▉         | 6861/70802 [00:14<05:23, 197.78it/s]

KeyboardInterrupt: 

In [224]:
seqs = test_df['seq'].to_numpy()
seqs = np.array([s for s in seqs])
onehot = test_df['onehot'].to_numpy()
onehot = np.array([v for v in onehot])

In [225]:
prob = model.predict([onehot, seqs], verbose=1)

70802/70802 [==============================] - 6s 78us/step


In [226]:
output_test_df = test_df.copy()

In [227]:
output_test_df['prob'] = prob.tolist()

 10%|▉         | 6861/70802 [00:27<05:23, 197.78it/s]

In [228]:
output_test_df.head()

,id,old_cc_no,seq,onehot,prob
0,105050,258419,"[[-0.1353477916210046, -1.1452132814152969, -0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.007855868898332119, 0.017383363097906113, 0..."
1,105050,258418,"[[1.7151820496364285, 0.023396338555622342, -0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.01289896946400404, 0.016550878062844276, 0...."
2,118056,279034,"[[-0.04616563059654999, -0.00396241508918179, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.00017050097812898457, 0.0011394848115742207..."
3,123561,186726,"[[0.10286245427326231, -0.12121421642405664, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.00987312477082014, 0.010647219605743885, 0...."
4,134768,138224,"[[-0.1353477916210046, -1.1452132814152969, -0...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.003235218347981572, 0.05584653839468956, 0...."


In [239]:
def ensemble(group):
#     group_prob = np.array(group['prob'].tolist()).sum(axis=0)
#     group_prob = softmax(group_prob)
    group_prob = np.array(group['prob'].tolist())[0]
    return group_prob

In [235]:
grouped = output_test_df.groupby('id')

In [240]:
dfs = []
for name, group in tqdm(grouped):
    class_prob = ensemble(group)
    data = {f'class{class_index}':p for class_index, p in enumerate(class_prob)}
    data['id'] = name
    dfs.append(pd.DataFrame(data, index=[name]))
output = pd.concat(dfs, ).set_index('id')


 41%|████▏     | 22060/53240 [00:28<00:40, 770.63it/s]


 83%|████████▎ | 43994/53240 [00:57<00:11, 773.18it/s]


100%|██████████| 53240/53240 [01:09<00:00, 766.06it/s]

In [241]:
output.to_csv('ensemble-first.csv', float_format='%.4f')